# Système de Trading Automatisé pour Bitget

Ce notebook permet d'exécuter un système de trading automatisé pour les futures USDT sur Bitget. Le système comprend :

- Sélection automatique des paires à trader
- Stratégie basée sur des indicateurs techniques avancés et machine learning
- Exécution directe via API
- Optimisation du risque/rendement
- Interface utilisateur intuitive

## Configuration initiale

Commençons par installer les dépendances nécessaires et configurer l'environnement.

In [ ]:
# Installation des dépendances
!pip install -q pandas numpy scipy matplotlib seaborn plotly scikit-learn xgboost tensorflow websocket-client requests ccxt ta-lib-binary pandas-ta ipywidgets

In [ ]:
# Cloner le dépôt du système de trading
!git clone https://github.com/username/bitget_trading_system.git
!cd bitget_trading_system && pip install -e .

In [ ]:
# Importer les bibliothèques nécessaires
import os
import sys
import time
import json
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime, timedelta
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Configurer le logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('bitget_trading_system')

## Configuration des clés API

Pour utiliser ce système, vous devez fournir vos clés API Bitget. Ces clés seront stockées de manière sécurisée dans des variables d'environnement.

In [ ]:
# Widgets pour la saisie des clés API
api_key_widget = widgets.Password(description='API Key:', placeholder='Entrez votre clé API Bitget', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
api_secret_widget = widgets.Password(description='API Secret:', placeholder='Entrez votre secret API Bitget', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))
passphrase_widget = widgets.Password(description='Passphrase:', placeholder='Entrez votre passphrase Bitget', style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))

display(api_key_widget, api_secret_widget, passphrase_widget)

def save_api_keys(button):
    os.environ["BITGET_API_KEY"] = api_key_widget.value
    os.environ["BITGET_API_SECRET"] = api_secret_widget.value
    os.environ["BITGET_PASSPHRASE"] = passphrase_widget.value
    print("✅ Clés API sauvegardées avec succès dans les variables d'environnement.")

save_button = widgets.Button(description="Sauvegarder les clés API", button_style='success')
save_button.on_click(save_api_keys)
display(save_button)

## Importation des modules du système de trading

Maintenant, importons les différents modules du système de trading.

In [ ]:
# Ajouter le chemin du système au PYTHONPATH
sys.path.append('./bitget_trading_system')

# Importer les modules du système de trading
from bitget_trading_system.code.pair_selection import PairSelector, BitgetDataFetcher
from bitget_trading_system.code.technical_indicators import IndicatorSet, create_default_indicator_set
from bitget_trading_system.code.ml_models import ModelEnsemble, create_default_model_ensemble
from bitget_trading_system.code.risk_management import RiskManager, create_default_risk_manager
from bitget_trading_system.code.api_integration import BitgetTrader, create_bitget_trader

print("✅ Modules importés avec succès.")

## Initialisation du système de trading

Initialisons maintenant les différents composants du système de trading.

In [ ]:
# Fonction pour initialiser le système de trading
def initialize_trading_system():
    # Récupérer les clés API
    api_key = os.environ.get("BITGET_API_KEY", "")
    api_secret = os.environ.get("BITGET_API_SECRET", "")
    passphrase = os.environ.get("BITGET_PASSPHRASE", "")
    
    if not api_key or not api_secret or not passphrase:
        print("❌ Erreur: Veuillez d'abord configurer vos clés API.")
        return None, None, None, None, None
    
    try:
        print("🔄 Initialisation du système de trading...")
        
        # Initialiser le trader Bitget
        trader = create_bitget_trader(api_key, api_secret, passphrase)
        print("✅ Trader Bitget initialisé.")
        
        # Initialiser le sélecteur de paires
        pair_selector = PairSelector(api_key, api_secret, passphrase)
        print("✅ Sélecteur de paires initialisé.")
        
        # Initialiser l'ensemble d'indicateurs techniques
        indicator_set = create_default_indicator_set()
        print("✅ Ensemble d'indicateurs techniques initialisé.")
        
        # Initialiser l'ensemble de modèles ML
        model_ensemble = create_default_model_ensemble()
        print("✅ Ensemble de modèles ML initialisé.")
        
        # Initialiser le gestionnaire de risque
        # Récupérer le solde du compte
        account_balance = trader.get_account_balance()
        risk_manager = create_default_risk_manager(account_balance)
        print(f"✅ Gestionnaire de risque initialisé avec un capital de {account_balance:.2f} USDT.")
        
        print("✅ Système de trading initialisé avec succès.")
        return trader, pair_selector, indicator_set, model_ensemble, risk_manager
    
    except Exception as e:
        print(f"❌ Erreur lors de l'initialisation du système: {str(e)}")
        return None, None, None, None, None

# Bouton pour initialiser le système
init_button = widgets.Button(description="Initialiser le système", button_style='info')

# Variables globales pour stocker les composants du système
trader = None
pair_selector = None
indicator_set = None
model_ensemble = None
risk_manager = None

def on_init_button_clicked(button):
    global trader, pair_selector, indicator_set, model_ensemble, risk_manager
    trader, pair_selector, indicator_set, model_ensemble, risk_manager = initialize_trading_system()

init_button.on_click(on_init_button_clicked)
display(init_button)

## Sélection des paires à trader

Utilisons l'algorithme de sélection des paires pour identifier les meilleures opportunités de trading.

In [ ]:
# Fonction pour sélectionner les paires à trader
def select_trading_pairs():
    global pair_selector
    
    if pair_selector is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return []
    
    try:
        print("🔄 Sélection des meilleures paires à trader...")
        top_pairs = pair_selector.select_pairs()
        
        # Afficher les résultats
        print("\n✅ Top paires à trader:")
        print("-" * 80)
        print(f"{'Symbol':<15} {'Score':<10} {'Direction':<10} {'Prix':<10} {'Vol 24h ($M)':<15} {'Volatilité':<10} {'Momentum':<10}")
        print("-" * 80)
        
        for pair in top_pairs:
            metrics = pair["metrics"]
            print(f"{pair['symbol']:<15} {pair['score']:.4f}    {pair['signal_direction']:<10} "
                  f"{metrics['current_price']:<10.4f} {metrics['volume_usd']/1000000:<15.2f} "
                  f"{metrics['volatility']:<10.2f} {metrics['momentum_24h']:<10.2f}")
        
        return top_pairs
    
    except Exception as e:
        print(f"❌ Erreur lors de la sélection des paires: {str(e)}")
        return []

# Bouton pour sélectionner les paires
select_pairs_button = widgets.Button(description="Sélectionner les paires", button_style='primary')
selected_pairs = []

def on_select_pairs_button_clicked(button):
    global selected_pairs
    selected_pairs = select_trading_pairs()

select_pairs_button.on_click(on_select_pairs_button_clicked)
display(select_pairs_button)

## Analyse technique et génération de signaux

Analysons les paires sélectionnées avec nos indicateurs techniques et modèles ML pour générer des signaux de trading.

In [ ]:
# Fonction pour analyser une paire et générer des signaux
def analyze_pair(symbol):
    global trader, indicator_set, model_ensemble
    
    if trader is None or indicator_set is None or model_ensemble is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return None, None
    
    try:
        print(f"🔄 Analyse de {symbol}...")
        
        # Récupérer les données historiques
        df = trader.get_historical_candles(symbol, "15m", 200)
        
        if df.empty:
            print(f"❌ Erreur: Impossible de récupérer les données pour {symbol}.")
            return None, None
        
        # Calculer les indicateurs techniques
        df_with_indicators = indicator_set.calculate_all(df)
        
        # Générer les signaux des indicateurs
        indicator_signals = indicator_set.generate_signals(df_with_indicators)
        
        # Générer un signal combiné à partir des indicateurs
        combined_indicator_signal = indicator_set.generate_combined_signal(df_with_indicators)
        
        # Générer un signal à partir des modèles ML
        ml_signal = model_ensemble.predict(df_with_indicators)
        
        # Combiner les signaux des indicateurs et des modèles ML
        final_signal = pd.Series(0, index=df_with_indicators.index)
        final_signal[combined_indicator_signal == 1] += 0.5
        final_signal[ml_signal == 1] += 0.5
        final_signal[combined_indicator_signal == -1] -= 0.5
        final_signal[ml_signal == -1] -= 0.5
        
        # Discrétiser le signal final
        discretized_signal = pd.Series(0, index=final_signal.index)
        discretized_signal[final_signal > 0.3] = 1
        discretized_signal[final_signal < -0.3] = -1
        
        # Ajouter les signaux au DataFrame
        df_with_indicators['indicator_signal'] = combined_indicator_signal
        df_with_indicators['ml_signal'] = ml_signal
        df_with_indicators['final_signal'] = discretized_signal
        
        # Obtenir le dernier signal
        last_signal = discretized_signal.iloc[-1] if not discretized_signal.empty else 0
        signal_str = "ACHAT" if last_signal == 1 else "VENTE" if last_signal == -1 else "NEUTRE"
        
        print(f"✅ Analyse terminée. Signal actuel: {signal_str}")
        
        return df_with_indicators, last_signal
    
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse de {symbol}: {str(e)}")
        return None, None

# Fonction pour visualiser les résultats de l'analyse
def visualize_analysis(df, symbol):
    if df is None or df.empty:
        print("❌ Erreur: Aucune donnée à visualiser.")
        return
    
    try:
        # Créer un graphique interactif avec Plotly
        fig = go.Figure()
        
        # Ajouter les chandeliers
        fig.add_trace(go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Prix'
        ))
        
        # Ajouter les moyennes mobiles si disponibles
        if 'ema_20' in df.columns:
            fig.add_trace(go.Scatter(
                x=df.index,
                y=df['ema_20'],
                name='EMA 20',
                line=dict(color='blue', width=1)
            ))
        
        if 'sma_200' in df.columns:
            fig.add_trace(go.Scatter(
                x=df.index,
                y=df['sma_200'],
                name='SMA 200',
                line=dict(color='red', width=1)
            ))
        
        # Ajouter les signaux d'achat et de vente
        if 'final_signal' in df.columns:
            buy_signals = df[df['final_signal'] == 1]
            sell_signals = df[df['final_signal'] == -1]
            
            fig.add_trace(go.Scatter(
                x=buy_signals.index,
                y=buy_signals['close'],
                mode='markers',
                marker=dict(color='green', size=10, symbol='triangle-up'),
                name='Signal d\'achat'
            ))
            
            fig.add_trace(go.Scatter(
                x=sell_signals.index,
                y=sell_signals['close'],
                mode='markers',
                marker=dict(color='red', size=10, symbol='triangle-down'),
                name='Signal de vente'
            ))
        
        # Configurer le layout
        fig.update_layout(
            title=f'Analyse technique de {symbol}',
            xaxis_title='Date',
            yaxis_title='Prix',
            xaxis_rangeslider_visible=False,
            height=600,
            template='plotly_dark'
        )
        
        # Afficher le graphique
        fig.show()
        
        # Afficher les indicateurs techniques dans un second graphique
        fig2 = go.Figure()
        
        # Ajouter le RSI si disponible
        if 'rsi' in df.columns:
            fig2.add_trace(go.Scatter(
                x=df.index,
                y=df['rsi'],
                name='RSI',
                line=dict(color='purple', width=1)
            ))
            
            # Ajouter les lignes de surachat/survente
            fig2.add_shape(type="line", x0=df.index[0], x1=df.index[-1], y0=70, y1=70,
                          line=dict(color="red", width=1, dash="dash"))
            fig2.add_shape(type="line", x0=df.index[0], x1=df.index[-1], y0=30, y1=30,
                          line=dict(color="green", width=1, dash="dash"))
        
        # Ajouter le MACD si disponible
        if all(col in df.columns for col in ['macd_line', 'macd_signal', 'macd_histogram']):
            fig3 = go.Figure()
            
            fig3.add_trace(go.Scatter(
                x=df.index,
                y=df['macd_line'],
                name='MACD Line',
                line=dict(color='blue', width=1)
            ))
            
            fig3.add_trace(go.Scatter(
                x=df.index,
                y=df['macd_signal'],
                name='Signal Line',
                line=dict(color='red', width=1)
            ))
            
            # Ajouter l'histogramme MACD
            colors = ['green' if val >= 0 else 'red' for val in df['macd_histogram']]
            fig3.add_trace(go.Bar(
                x=df.index,
                y=df['macd_histogram'],
                name='Histogramme',
                marker_color=colors
            ))
            
            fig3.update_layout(
                title='MACD',
                xaxis_title='Date',
                yaxis_title='Valeur',
                height=300,
                template='plotly_dark'
            )
            
            fig3.show()
        
        # Configurer le layout du second graphique
        fig2.update_layout(
            title='RSI',
            xaxis_title='Date',
            yaxis_title='Valeur',
            height=300,
            template='plotly_dark'
        )
        
        # Afficher le second graphique
        fig2.show()
        
    except Exception as e:
        print(f"❌ Erreur lors de la visualisation: {str(e)}")

# Dropdown pour sélectionner une paire à analyser
pair_dropdown = widgets.Dropdown(
    options=[],
    description='Paire:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Fonction pour mettre à jour le dropdown avec les paires sélectionnées
def update_pair_dropdown():
    global selected_pairs, pair_dropdown
    pair_dropdown.options = [(pair["symbol"], pair["symbol"]) for pair in selected_pairs]

# Bouton pour analyser la paire sélectionnée
analyze_button = widgets.Button(description="Analyser la paire", button_style='warning')

def on_analyze_button_clicked(button):
    symbol = pair_dropdown.value
    if not symbol:
        print("❌ Erreur: Veuillez sélectionner une paire à analyser.")
        return
    
    df, signal = analyze_pair(symbol)
    if df is not None:
        visualize_analysis(df, symbol)

analyze_button.on_click(on_analyze_button_clicked)

# Bouton pour mettre à jour le dropdown
update_dropdown_button = widgets.Button(description="Mettre à jour la liste des paires", button_style='info')

def on_update_dropdown_button_clicked(button):
    update_pair_dropdown()

update_dropdown_button.on_click(on_update_dropdown_button_clicked)

# Afficher les widgets
display(update_dropdown_button, pair_dropdown, analyze_button)

## Gestion du risque et calcul des paramètres de trading

Utilisons notre système de gestion du risque pour calculer les paramètres optimaux pour les trades.

In [ ]:
# Fonction pour calculer les paramètres de trading
def calculate_trade_parameters(symbol, signal):
    global trader, risk_manager
    
    if trader is None or risk_manager is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return None
    
    if signal == 0:
        print("ℹ️ Signal neutre, aucun paramètre de trading à calculer.")
        return None
    
    try:
        print(f"🔄 Calcul des paramètres de trading pour {symbol}...")
        
        # Récupérer les données nécessaires
        df = trader.get_historical_candles(symbol, "15m", 20)
        
        if df.empty:
            print(f"❌ Erreur: Impossible de récupérer les données pour {symbol}.")
            return None
        
        # Calculer l'ATR
        df['tr'] = np.maximum(
            df['high'] - df['low'],
            np.maximum(
                np.abs(df['high'] - df['close'].shift(1)),
                np.abs(df['low'] - df['close'].shift(1))
            )
        )
        atr = df['tr'].mean()
        
        # Récupérer le prix actuel
        current_price = df['close'].iloc[-1]
        
        # Calculer la volatilité
        volatility = df['close'].pct_change().std() * np.sqrt(20)
        volatility_factor = volatility / 0.03  # Normaliser par rapport à une volatilité de référence de 3%
        
        # Déterminer le type de position
        position_type = "long" if signal == 1 else "short"
        
        # Récupérer les positions actuelles
        positions = trader.get_positions()
        
        # Calculer les paramètres de trading
        trade_params = risk_manager.calculate_trade_parameters(
            symbol=symbol,
            entry_price=current_price,
            position_type=position_type,
            atr=atr,
            volatility_factor=volatility_factor,
            signal_strength=0.7,  # Valeur arbitraire, pourrait être calculée à partir de la confiance du signal
            market_regime="normal"  # Pourrait être déterminé par un clusterer de régimes de marché
        )
        
        print("✅ Paramètres de trading calculés avec succès.")
        
        # Afficher les paramètres
        print("\nParamètres de trading:")
        print(f"Symbol: {trade_params['symbol']}")
        print(f"Type de position: {trade_params['position_type']}")
        print(f"Prix d'entrée: {trade_params['entry_price']:.4f}")
        print(f"Stop loss: {trade_params['stop_loss']:.4f}")
        print(f"Take profit: {', '.join([f'{tp:.4f}' for tp in trade_params['take_profit_levels']])}")
        print(f"Taille de la position: {trade_params['position_value']:.2f} USDT")
        print(f"Unités: {trade_params['units']:.6f}")
        print(f"Risque: {trade_params['risk_percentage']:.2f}% du capital")
        print(f"Levier: {trade_params['leverage']:.2f}x")
        
        return trade_params
    
    except Exception as e:
        print(f"❌ Erreur lors du calcul des paramètres de trading: {str(e)}")
        return None

# Bouton pour calculer les paramètres de trading
calc_params_button = widgets.Button(description="Calculer les paramètres de trading", button_style='warning')
trade_params = None

def on_calc_params_button_clicked(button):
    global trade_params
    symbol = pair_dropdown.value
    if not symbol:
        print("❌ Erreur: Veuillez sélectionner une paire à analyser.")
        return
    
    # Analyser la paire pour obtenir le signal
    _, signal = analyze_pair(symbol)
    
    if signal is not None:
        trade_params = calculate_trade_parameters(symbol, signal)

calc_params_button.on_click(on_calc_params_button_clicked)
display(calc_params_button)

## Exécution des ordres

Exécutons maintenant des ordres sur Bitget en utilisant les paramètres calculés.

In [ ]:
# Fonction pour exécuter un ordre
def execute_order(trade_params):
    global trader
    
    if trader is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return
    
    if trade_params is None:
        print("❌ Erreur: Veuillez d'abord calculer les paramètres de trading.")
        return
    
    try:
        symbol = trade_params['symbol']
        position_type = trade_params['position_type']
        units = trade_params['units']
        entry_price = trade_params['entry_price']
        leverage = trade_params['leverage']
        
        # Confirmation de l'utilisateur
        print(f"⚠️ Vous êtes sur le point d'exécuter un ordre {position_type.upper()} pour {units:.6f} unités de {symbol} au prix de {entry_price:.4f} avec un levier de {leverage:.2f}x.")
        confirm = input("Tapez 'CONFIRMER' pour exécuter l'ordre: ")
        
        if confirm != "CONFIRMER":
            print("❌ Ordre annulé.")
            return
        
        print(f"🔄 Exécution de l'ordre {position_type.upper()} pour {symbol}...")
        
        # Définir le levier
        leverage_result = trader.set_leverage(symbol, int(leverage))
        print(f"✅ Levier défini à {leverage:.2f}x.")
        
        # Déterminer le côté de l'ordre
        side = "buy" if position_type == "long" else "sell"
        
        # Exécuter l'ordre au marché
        order_result = trader.place_market_order(
            symbol=symbol,
            side=side,
            size=str(units),
            margin_coin="USDT"
        )
        
        print(f"✅ Ordre exécuté avec succès. ID de l'ordre: {order_result.get('orderId', 'N/A')}")
        
        # Récupérer la position
        time.sleep(2)  # Attendre que la position soit mise à jour
        position = trader.get_position(symbol)
        
        if position:
            print(f"\nPosition ouverte:")
            print(f"Symbol: {position.get('symbol', 'N/A')}")
            print(f"Type: {position.get('holdSide', 'N/A')}")
            print(f"Taille: {position.get('total', 'N/A')}")
            print(f"Prix moyen: {position.get('averageOpenPrice', 'N/A')}")
            print(f"Levier: {position.get('leverage', 'N/A')}x")
        
        return order_result
    
    except Exception as e:
        print(f"❌ Erreur lors de l'exécution de l'ordre: {str(e)}")
        return None

# Bouton pour exécuter un ordre
execute_button = widgets.Button(description="Exécuter l'ordre", button_style='danger')

def on_execute_button_clicked(button):
    global trade_params
    execute_order(trade_params)

execute_button.on_click(on_execute_button_clicked)
display(execute_button)

## Surveillance des positions

Surveillons les positions ouvertes et gérons-les.

In [ ]:
# Fonction pour afficher les positions ouvertes
def display_positions():
    global trader
    
    if trader is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return
    
    try:
        print("🔄 Récupération des positions ouvertes...")
        positions = trader.get_positions()
        
        if not positions:
            print("ℹ️ Aucune position ouverte.")
            return
        
        # Créer un DataFrame pour afficher les positions
        positions_data = []
        for pos in positions:
            if float(pos.get('total', 0)) > 0:
                positions_data.append({
                    'Symbol': pos.get('symbol', 'N/A'),
                    'Type': pos.get('holdSide', 'N/A'),
                    'Taille': float(pos.get('total', 0)),
                    'Prix moyen': float(pos.get('averageOpenPrice', 0)),
                    'Levier': float(pos.get('leverage', 0)),
                    'PnL': float(pos.get('unrealizedPL', 0)),
                    'PnL %': float(pos.get('unrealizedPL', 0)) / float(pos.get('margin', 1)) * 100 if float(pos.get('margin', 0)) > 0 else 0
                })
        
        if not positions_data:
            print("ℹ️ Aucune position ouverte.")
            return
        
        positions_df = pd.DataFrame(positions_data)
        display(positions_df)
        
        return positions_data
    
    except Exception as e:
        print(f"❌ Erreur lors de l'affichage des positions: {str(e)}")
        return None

# Fonction pour fermer une position
def close_position(symbol):
    global trader
    
    if trader is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return
    
    try:
        print(f"🔄 Fermeture de la position pour {symbol}...")
        result = trader.close_position(symbol)
        
        if result.get('success', False):
            print(f"✅ Position fermée avec succès.")
        else:
            print(f"❌ Erreur lors de la fermeture de la position: {result.get('message', 'Erreur inconnue')}")
        
        return result
    
    except Exception as e:
        print(f"❌ Erreur lors de la fermeture de la position: {str(e)}")
        return None

# Bouton pour afficher les positions
display_positions_button = widgets.Button(description="Afficher les positions", button_style='info')

def on_display_positions_button_clicked(button):
    display_positions()

display_positions_button.on_click(on_display_positions_button_clicked)

# Dropdown pour sélectionner une position à fermer
position_dropdown = widgets.Dropdown(
    options=[],
    description='Position:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Fonction pour mettre à jour le dropdown des positions
def update_position_dropdown():
    global trader, position_dropdown
    
    if trader is None:
        return
    
    try:
        positions = trader.get_positions()
        position_options = []
        
        for pos in positions:
            if float(pos.get('total', 0)) > 0:
                symbol = pos.get('symbol', 'N/A')
                position_options.append((symbol, symbol))
        
        position_dropdown.options = position_options
    
    except Exception as e:
        print(f"❌ Erreur lors de la mise à jour du dropdown des positions: {str(e)}")

# Bouton pour mettre à jour le dropdown des positions
update_position_dropdown_button = widgets.Button(description="Mettre à jour la liste des positions", button_style='info')

def on_update_position_dropdown_button_clicked(button):
    update_position_dropdown()

update_position_dropdown_button.on_click(on_update_position_dropdown_button_clicked)

# Bouton pour fermer une position
close_position_button = widgets.Button(description="Fermer la position", button_style='danger')

def on_close_position_button_clicked(button):
    symbol = position_dropdown.value
    if not symbol:
        print("❌ Erreur: Veuillez sélectionner une position à fermer.")
        return
    
    close_position(symbol)

close_position_button.on_click(on_close_position_button_clicked)

# Afficher les widgets
display(display_positions_button, update_position_dropdown_button, position_dropdown, close_position_button)

## Mode automatique

Activons le mode automatique pour que le système fonctionne sans intervention.

In [ ]:
# Variables globales pour le mode automatique
auto_mode_running = False
auto_mode_thread = None

# Fonction pour exécuter le mode automatique
def run_auto_mode(interval_minutes=15, max_positions=3, risk_per_trade=0.01):
    global auto_mode_running, trader, pair_selector, indicator_set, model_ensemble, risk_manager
    
    if trader is None or pair_selector is None or indicator_set is None or model_ensemble is None or risk_manager is None:
        print("❌ Erreur: Veuillez d'abord initialiser le système.")
        return
    
    auto_mode_running = True
    print(f"✅ Mode automatique activé. Intervalle: {interval_minutes} minutes, Positions max: {max_positions}, Risque par trade: {risk_per_trade*100}%")
    
    try:
        while auto_mode_running:
            try:
                # Étape 1: Sélectionner les paires
                print("\n🔄 Sélection des paires...")
                top_pairs = pair_selector.select_pairs()
                
                if not top_pairs:
                    print("❌ Aucune paire sélectionnée.")
                    time.sleep(interval_minutes * 60)
                    continue
                
                # Étape 2: Récupérer les positions actuelles
                positions = trader.get_positions()
                open_positions = [p for p in positions if float(p.get('total', 0)) > 0]
                
                # Vérifier si on peut ouvrir de nouvelles positions
                if len(open_positions) >= max_positions:
                    print(f"ℹ️ Nombre maximum de positions atteint ({len(open_positions)}/{max_positions}).")
                else:
                    # Étape 3: Analyser les paires et générer des signaux
                    for pair in top_pairs:
                        symbol = pair['symbol']
                        
                        # Vérifier si on a déjà une position sur cette paire
                        if any(p.get('symbol') == symbol for p in open_positions):
                            print(f"ℹ️ Position déjà ouverte sur {symbol}, ignoré.")
                            continue
                        
                        print(f"\n🔄 Analyse de {symbol}...")
                        df, signal = analyze_pair(symbol)
                        
                        if signal == 0:
                            print(f"ℹ️ Signal neutre pour {symbol}, ignoré.")
                            continue
                        
                        # Étape 4: Calculer les paramètres de trading
                        trade_params = calculate_trade_parameters(symbol, signal)
                        
                        if trade_params is None:
                            print(f"❌ Impossible de calculer les paramètres de trading pour {symbol}, ignoré.")
                            continue
                        
                        # Ajuster le risque par trade
                        original_risk = trade_params['risk_percentage'] / 100
                        risk_ratio = risk_per_trade / original_risk
                        trade_params['units'] *= risk_ratio
                        trade_params['position_value'] *= risk_ratio
                        trade_params['risk_amount'] *= risk_ratio
                        trade_params['risk_percentage'] = risk_per_trade * 100
                        
                        # Étape 5: Exécuter l'ordre
                        print(f"\n🔄 Exécution de l'ordre {trade_params['position_type'].upper()} pour {symbol}...")
                        
                        # Définir le levier
                        leverage_result = trader.set_leverage(symbol, int(trade_params['leverage']))
                        
                        # Déterminer le côté de l'ordre
                        side = "buy" if trade_params['position_type'] == "long" else "sell"
                        
                        # Exécuter l'ordre au marché
                        order_result = trader.place_market_order(
                            symbol=symbol,
                            side=side,
                            size=str(trade_params['units']),
                            margin_coin="USDT"
                        )
                        
                        print(f"✅ Ordre exécuté avec succès. ID de l'ordre: {order_result.get('orderId', 'N/A')}")
                        
                        # Mettre à jour le nombre de positions ouvertes
                        open_positions = trader.get_positions()
                        open_positions = [p for p in open_positions if float(p.get('total', 0)) > 0]
                        
                        if len(open_positions) >= max_positions:
                            print(f"ℹ️ Nombre maximum de positions atteint ({len(open_positions)}/{max_positions}).")
                            break
                
                # Étape 6: Gérer les positions existantes
                if open_positions:
                    print("\n🔄 Gestion des positions existantes...")
                    
                    for position in open_positions:
                        symbol = position.get('symbol')
                        position_type = position.get('holdSide')
                        entry_price = float(position.get('averageOpenPrice', 0))
                        pnl_pct = float(position.get('unrealizedPL', 0)) / float(position.get('margin', 1)) * 100 if float(position.get('margin', 0)) > 0 else 0
                        
                        print(f"\nPosition sur {symbol} ({position_type}): PnL = {pnl_pct:.2f}%")
                        
                        # Analyser la paire pour voir si le signal a changé
                        df, signal = analyze_pair(symbol)
                        
                        # Vérifier si le signal est inversé
                        signal_inversed = (position_type == "long" and signal == -1) or (position_type == "short" and signal == 1)
                        
                        # Fermer la position si le signal est inversé ou si le PnL est supérieur à 5%
                        if signal_inversed or pnl_pct > 5.0 or pnl_pct < -3.0:
                            reason = "signal inversé" if signal_inversed else "prise de profit" if pnl_pct > 5.0 else "stop loss"
                            print(f"🔄 Fermeture de la position sur {symbol} ({reason})...")
                            close_result = trader.close_position(symbol)
                            print(f"✅ Position fermée avec succès.")
                
                # Attendre jusqu'au prochain intervalle
                print(f"\n⏱️ Attente de {interval_minutes} minutes jusqu'au prochain cycle...")
                for i in range(interval_minutes):
                    if not auto_mode_running:
                        break
                    time.sleep(60)
            
            except Exception as e:
                print(f"❌ Erreur dans le mode automatique: {str(e)}")
                time.sleep(60)  # Attendre 1 minute en cas d'erreur
    
    except KeyboardInterrupt:
        print("\n⚠️ Mode automatique interrompu par l'utilisateur.")
    
    auto_mode_running = False
    print("✅ Mode automatique désactivé.")

# Widgets pour configurer le mode automatique
interval_slider = widgets.IntSlider(
    value=15,
    min=5,
    max=60,
    step=5,
    description='Intervalle (min):',
    style={'description_width': 'initial'}
)

max_positions_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Positions max:',
    style={'description_width': 'initial'}
)

risk_slider = widgets.FloatSlider(
    value=0.01,
    min=0.005,
    max=0.05,
    step=0.005,
    description='Risque par trade:',
    style={'description_width': 'initial'},
    readout_format='.1%'
)

# Bouton pour démarrer le mode automatique
start_auto_button = widgets.Button(description="Démarrer le mode automatique", button_style='success')

def on_start_auto_button_clicked(button):
    global auto_mode_running, auto_mode_thread
    
    if auto_mode_running:
        print("⚠️ Le mode automatique est déjà en cours d'exécution.")
        return
    
    auto_mode_thread = threading.Thread(
        target=run_auto_mode,
        args=(interval_slider.value, max_positions_slider.value, risk_slider.value)
    )
    auto_mode_thread.daemon = True
    auto_mode_thread.start()

start_auto_button.on_click(on_start_auto_button_clicked)

# Bouton pour arrêter le mode automatique
stop_auto_button = widgets.Button(description="Arrêter le mode automatique", button_style='danger')

def on_stop_auto_button_clicked(button):
    global auto_mode_running
    
    if not auto_mode_running:
        print("⚠️ Le mode automatique n'est pas en cours d'exécution.")
        return
    
    auto_mode_running = False
    print("⏱️ Arrêt du mode automatique en cours...")

stop_auto_button.on_click(on_stop_auto_button_clicked)

# Afficher les widgets
display(widgets.HBox([interval_slider, max_positions_slider, risk_slider]))
display(widgets.HBox([start_auto_button, stop_auto_button]))

## Nettoyage et fermeture

Nettoyons les ressources utilisées par le système.

In [ ]:
# Fonction pour nettoyer les ressources
def cleanup():
    global trader, auto_mode_running
    
    # Arrêter le mode automatique
    auto_mode_running = False
    
    # Nettoyer les ressources du trader
    if trader is not None:
        trader.cleanup()
    
    print("✅ Ressources nettoyées avec succès.")

# Bouton pour nettoyer les ressources
cleanup_button = widgets.Button(description="Nettoyer les ressources", button_style='warning')

def on_cleanup_button_clicked(button):
    cleanup()

cleanup_button.on_click(on_cleanup_button_clicked)
display(cleanup_button)

## Conclusion

Félicitations ! Vous avez maintenant un système de trading automatisé complet pour Bitget. Ce système comprend :

1. **Sélection automatique des paires** - Identifie dynamiquement les meilleures opportunités de trading
2. **Analyse technique avancée** - Utilise des indicateurs techniques et des modèles de machine learning
3. **Gestion du risque adaptative** - Ajuste dynamiquement les paramètres de trading en fonction des conditions du marché
4. **Exécution directe via API** - Exécute les ordres directement sur Bitget
5. **Interface utilisateur intuitive** - Permet de contrôler facilement le système

N'oubliez pas que le trading comporte des risques. Utilisez ce système avec prudence et ne tradez jamais plus que ce que vous pouvez vous permettre de perdre.